## Part B - Event Producer 1 

Name: Ko Ko Win <br> 
ID : 31842305

In [5]:
import pymongo 
from pymongo import MongoClient
from pprint import pprint
client =  MongoClient()
db = client.fit3182_assignment_db
collection = db.climate

In [9]:
#Querying latest date
results = collection.find({},{"date":1, "_id":0})
for res in results: 
    empt = []
    new_date = ({"date" : res["date"].strftime("%d/%m/%Y")})
    empt.append(new_date)
    #latest date it at bottom most
    pprint(sorted(empt))

[{'date': '31/12/2020'}]
[{'date': '02/01/2021'}]
[{'date': '03/01/2021'}]
[{'date': '04/01/2021'}]
[{'date': '05/01/2021'}]
[{'date': '06/01/2021'}]
[{'date': '07/01/2021'}]
[{'date': '08/01/2021'}]
[{'date': '09/01/2021'}]
[{'date': '10/01/2021'}]
[{'date': '11/01/2021'}]
[{'date': '12/01/2021'}]
[{'date': '13/01/2021'}]
[{'date': '14/01/2021'}]
[{'date': '15/01/2021'}]
[{'date': '16/01/2021'}]
[{'date': '17/01/2021'}]
[{'date': '18/01/2021'}]
[{'date': '19/01/2021'}]
[{'date': '20/01/2021'}]
[{'date': '21/01/2021'}]
[{'date': '22/01/2021'}]
[{'date': '23/01/2021'}]
[{'date': '24/01/2021'}]
[{'date': '25/01/2021'}]
[{'date': '26/01/2021'}]
[{'date': '27/01/2021'}]
[{'date': '28/01/2021'}]
[{'date': '29/01/2021'}]
[{'date': '30/01/2021'}]
[{'date': '31/01/2021'}]
[{'date': '01/02/2021'}]
[{'date': '02/02/2021'}]
[{'date': '03/02/2021'}]
[{'date': '04/02/2021'}]
[{'date': '05/02/2021'}]
[{'date': '06/02/2021'}]
[{'date': '07/02/2021'}]
[{'date': '08/02/2021'}]
[{'date': '09/02/2021'}]


In [2]:
import pandas as pd 
from time import sleep 
from json import dumps 
from kafka import KafkaProducer 
import random 
import datetime as dt 
import pymongo 
from pymongo import MongoClient 
from pprint import pprint
import json

def read_data(): 
    climate_streaming = pd.read_csv("climate_streaming.csv")  #read csv data 
    climate_streaming.columns = climate_streaming.columns.str.replace(' ', '') #remove whitespaces from column name
    climate_streaming["precipitation_flag"] = climate_streaming["precipitation"].str[-1] #get precipitation flag
    climate_streaming["precipitation_value"] = climate_streaming["precipitation"].str[0:5] #get precipitation value
    
    climate_array = []
    
    for index, row in climate_streaming.iterrows(): 
        cs_docs = {}
        cs_docs["latitude"] = float(row["latitude"])
        cs_docs["longitude"] = float(row["longitude"]) 
        cs_docs["air_temperature_celcius"] = int(row["air_temperature_celcius"]) 
        cs_docs["relative_humidity"] = float(row["relative_humidity"]) 
        cs_docs["windspeed_knots"] = float(row["windspeed_knots"]) 
        cs_docs["max_wind_speed"] = float(row["max_wind_speed"]) 
        cs_docs["precipitation"] = str(row["precipitation"]) 
        cs_docs["GHI_w/m2"] = int(row["GHI_w/m2"]) 
        cs_docs["precipitation_flag"] = str(row["precipitation_flag"]) 
        cs_docs["precipitation_value"] = float(row["precipitation_value"])
        
        #append document to climate array
        climate_array.append(cs_docs)
        
    
    return climate_array


def publish_message(producer_instance, topic_name, key, data):
    try:
        str_value = json.dumps(data, indent =3) #convert JSON to bytes 
        key_bytes = bytes(key, encoding="utf-8") #cast key to bytes
        value_bytes = bytes(str_value, encoding="utf-8") #cast value to bytes 
        #send key-value bytes to kafka topic (topic_name)
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str_value) 
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    


if __name__ == '__main__':
    topic = "Producer01" 
    print('Publishing records..')
    producer = connect_kafka_producer()
    created_date = dt.datetime(2021, 12, 31, 0, 0)
    new_date = created_date + dt.timedelta(days=1) #Increment day by one
    cs_data = read_data() 
    
    while True:
        random_num = random.randrange(0, len(cs_data)) #generate random integer
        data = cs_data[random_num] #access the climate streaming based on the random_num 
        time  = dt.datetime.now().strftime("%H:%M:%S") #find the time being sent 
        data["time"] = time #append time info in data
        data["created_date"] = new_date.strftime("%d/%m/%Y") #store the date created as string 
        data["producer_information"] = "Event Producer 1 (Climate)" #append producer info
        publish_message(producer, topic, 'parsed', data)
        new_date = new_date + dt.timedelta(days=1) #add new day after every 10 secs
        
        #send every 10 seconds 
        sleep(10)
        
    
    

Publishing records..
Message published successfully. Data: {
   "latitude": -36.009,
   "longitude": 143.185,
   "air_temperature_celcius": 28,
   "relative_humidity": 48.3,
   "windspeed_knots": 11.9,
   "max_wind_speed": 22.0,
   "precipitation": " 0.00I",
   "GHI_w/m2": 243,
   "precipitation_flag": "I",
   "precipitation_value": 0.0,
   "time": "12:45:03",
   "created_date": "01/01/2022",
   "producer_information": "Event Producer 1 (Climate)"
}
Message published successfully. Data: {
   "latitude": -37.475,
   "longitude": 143.365,
   "air_temperature_celcius": 17,
   "relative_humidity": 49.9,
   "windspeed_knots": 12.2,
   "max_wind_speed": 21.0,
   "precipitation": " 0.24G",
   "GHI_w/m2": 146,
   "precipitation_flag": "G",
   "precipitation_value": 0.24,
   "time": "12:45:13",
   "created_date": "02/01/2022",
   "producer_information": "Event Producer 1 (Climate)"
}
Message published successfully. Data: {
   "latitude": -37.758,
   "longitude": 148.721,
   "air_temperature_cel

KeyboardInterrupt: 